In [1]:
import sys

In [2]:
sys.path.append('/scratch/pradap/python-work/enrique/')

In [3]:
import magellan as mg

In [4]:
A = mg.load_dataset('table_A')
B = mg.load_dataset('table_B')

In [5]:
ab = mg.AttrEquivalenceBlocker()
C = ab.block_tables(A, B, 'zipcode', 'zipcode', ['name'], ['name'])

In [6]:
#L = mg.read_csv('label_demo.csv', ltable=A, rtable=B)
#L = mg.label_table(C, 'gold')

In [7]:
L = mg.read_csv('lable_ab_correct_labels.csv', ltable=A, rtable=B)

In [8]:
mg.init_jvm()

True

In [9]:
feat_table = mg.get_features_for_matching(A, B)

In [10]:
f = feat_table.ix[[9,10, 17]]

In [11]:
G = mg.extract_feat_vecs(L, feat_table=f, attrs_after='gold')

In [12]:
G

,_id,ltable.ID,rtable.ID,name_name_mel,name_name_lev,hourly_wage_hourly_wage_exm,gold
0,0,a1,b1,0.327273,0.000000,0,0
1,1,a1,b2,0.145455,0.090909,0,0
2,2,a1,b6,0.218182,0.071429,0,0
3,3,a3,b1,0.145455,0.142857,0,0
4,4,a3,b2,0.781818,0.714286,1,1
5,5,a3,b6,0.400000,0.428571,0,0
6,6,a2,b3,0.707692,0.750000,1,1
7,7,a2,b4,0.181818,0.187500,0,0
8,8,a2,b5,0.169231,0.062500,0,0
9,9,a4,b3,0.166667,0.076923,0,0


In [13]:
dt = mg.DTMatcher()
svm = mg.SVMMatcher()
rf = mg.RFMatcher()
nb = mg.NBMatcher()
ln = mg.LinRegMatcher()
lg = mg.LogRegMatcher()



In [14]:
m, stats = mg.select_matcher([dt, svm, rf, nb, ln, lg], table = G, exclude_attrs=['_id', 'ltable.ID', 'rtable.ID', 'gold'], target_attr=['gold'], metric='f1', k=5)

/scratch/pradap/local/share/anaconda/lib/python2.7/site-packages/sklearn/metrics/classification.py:958: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [15]:
stats

,Name,Matcher,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Mean score
0,DecisionTree_80602239982632365441,<magellan.matcher.dtmatcher.DTMatcher object a...,5,1,1,0,0,0,0.4
1,SVM_82391049982632365441,<magellan.matcher.svmmatcher.SVMMatcher object...,5,1,0,0,0,0,0.2
2,RandomForest_23692649982632365441,<magellan.matcher.rfmatcher.RFMatcher object a...,5,1,1,0,0,0,0.4
3,NaiveBayes_80891359982632365441,<magellan.matcher.nbmatcher.NBMatcher object a...,5,1,1,0,0,0,0.4
4,LinearRegression_82106759982632365441,<magellan.matcher.linregmatcher.LinRegMatcher ...,5,1,1,0,0,0,0.4
5,LogisticRegression_08881269982632365441,<magellan.matcher.logregmatcher.LogRegMatcher ...,5,1,0,0,0,0,0.2


In [16]:
dt.fit(table=G,  exclude_attrs=['_id', 'ltable.ID', 'rtable.ID', 'gold'], target_attr='gold')

In [17]:
mg.debug_decisiontree_matcher(dt, A.ix[1], B.ix[3], f, G.columns,['_id', 'ltable.ID', 'rtable.ID', 'gold'])

def debug_fn(): 
    if ( name_name_mel <= 0.553846180439 ):
        print '    name_name_mel <= 0.553846180439 is True (  value : '  + str(name_name_mel) + ')'
        return False #( 12 examples )
    else:
        print '    name_name_mel <= 0.553846180439 is False (  value : '  + str(name_name_mel) + ')'
        return True #( 3 examples )

    name_name_mel <= 0.553846180439 is True (  value : 0.181818181818)
Match status : False


In [18]:
dt.clf.tree_